In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
spark = SparkSession\
    .builder \
    .appName("Kmeans Clustering Lab") \
    .getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

We are reading data from the csv file and putting it into a Spark dataframe and letting it infer the schema from the header.

In [6]:
df = spark.read.csv('s3://cs496-proga1-storage/covid_data/metadata.csv',
                    sep=',',
                    inferSchema=True,
                    header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- cord_uid: string (nullable = true)
 |-- sha: string (nullable = true)
 |-- source_x: string (nullable = true)
 |-- title: string (nullable = true)
 |-- doi: string (nullable = true)
 |-- pmcid: string (nullable = true)
 |-- pubmed_id: string (nullable = true)
 |-- license: string (nullable = true)
 |-- abstract: string (nullable = true)
 |-- publish_time: string (nullable = true)
 |-- authors: string (nullable = true)
 |-- journal: string (nullable = true)
 |-- mag_id: string (nullable = true)
 |-- who_covidence_id: string (nullable = true)
 |-- arxiv_id: string (nullable = true)
 |-- pdf_json_files: string (nullable = true)
 |-- pmc_json_files: string (nullable = true)
 |-- url: string (nullable = true)
 |-- s2_id: string (nullable = true)

We only want to keep the tile and abstract columns. We drop the rest from the dataframe.

In [17]:
df = df.drop('cord_uid', 'sha', 'source_x', 'doi', 'pmcid', 'pubmed_id', 'license', 'publish_time', 'authors', 'journal', 'mag_id', 'who_covidence_id', 'arxiv_id', 'pdf_json_files', 'pmc_json_files', 'url', 's2_id')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+
|               title|            abstract|
+--------------------+--------------------+
|Clinical features...|OBJECTIVE: This r...|
|Nitric oxide: a p...|Inflammatory dise...|
|Surfactant protei...|Surfactant protei...|
|Role of endotheli...|Endothelin-1 (ET-...|
|Gene expression i...|Respiratory syncy...|
|Sequence requirem...|Nidovirus subgeno...|
|Debate: Transfusi...|Recent evidence s...|
|The 21st Internat...|The 21st Internat...|
|Heme oxygenase-1 ...|Heme oxygenase-1 ...|
|Technical Descrip...|This report descr...|
|Conservation of p...|Regulation of orn...|
|Heterogeneous nuc...|Heterogeneous nuc...|
|A Method to Ident...|The UBA domain is...|
|Vaccinia virus in...|We examined the r...|
|The site of origi...|                null|
|Multi-faceted, mu...|There are hundred...|
|Herpes simplex vi...|INTRODUCTION: The...|
|Logistics of comm...|BACKGROUND: Previ...|
|Protection of pul...|BACKGROUND: Oxyge...|
|Bioinformatic map...|BACKGROUND

We drop the null values from the dataframe.

In [19]:
df = df.na.drop()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+
|               title|            abstract|
+--------------------+--------------------+
|Clinical features...|OBJECTIVE: This r...|
|Nitric oxide: a p...|Inflammatory dise...|
|Surfactant protei...|Surfactant protei...|
|Role of endotheli...|Endothelin-1 (ET-...|
|Gene expression i...|Respiratory syncy...|
|Sequence requirem...|Nidovirus subgeno...|
|Debate: Transfusi...|Recent evidence s...|
|The 21st Internat...|The 21st Internat...|
|Heme oxygenase-1 ...|Heme oxygenase-1 ...|
|Technical Descrip...|This report descr...|
|Conservation of p...|Regulation of orn...|
|Heterogeneous nuc...|Heterogeneous nuc...|
|A Method to Ident...|The UBA domain is...|
|Vaccinia virus in...|We examined the r...|
|Multi-faceted, mu...|There are hundred...|
|Herpes simplex vi...|INTRODUCTION: The...|
|Logistics of comm...|BACKGROUND: Previ...|
|Protection of pul...|BACKGROUND: Oxyge...|
|Bioinformatic map...|BACKGROUND: AlkB-...|
|Managing emerging...|In the 198

Use vector assembler to turn columns into an array. Source: https://rsandstroem.github.io/sparkkmeans.html

In [ ]:
v_assembler = VectorAssembler(inputCols